In [1]:
import xarray as xr
import arcpy
from arcpy.sa import *
import uuid

in_cube = r"C:\Users\Lewis\Desktop\test\cube.nc"

ds = xr.open_dataset(in_cube)
ds= ds.load()
ds.close()

ds =  (ds['nbart_nir'] - ds['nbart_red']) / (ds['nbart_nir'] + ds['nbart_red'])
ds = ds.to_dataset(name='veg_idx')

ds = xr.where(ds['veg_idx'] > 0.6, 1, 0)

ds = ds.sum('time')

df = ds.to_dataframe().to_records()

arcpy.env.addOutputsToMap = False

out_fc = 'temp'
srs = arcpy.SpatialReference(3577)
tmp_feat = arcpy.management.CreateFeatureclass(out_path='in_memory', 
                                               out_name=out_fc, 
                                               geometry_type='POINT', 
                                               spatial_reference=srs)

arcpy.AddField_management(in_table=tmp_feat, 
                          field_name='samples', 
                          field_type='LONG')

fields = ['samples', 'SHAPE@XY']
with arcpy.da.InsertCursor(tmp_feat, fields) as cursor:
    for point in df:
        row = [point[3], (point[1], point[0])]
        cursor.insertRow(row)
        

try:
    if arcpy.CheckExtension('Spatial') == 'Available':
        arcpy.CheckOutExtension('Spatial')
        
        arcpy.env.addOutputsToMap = True
        kdense = KernelDensity(in_features=tmp_feat, 
                               population_field='samples', 
                               search_radius=30,
                               area_unit_scale_factor='SQUARE_METERS')

        arcpy.CheckInExtension('Spatial')
        
    else:
        arcpy.AddError('Spatial Analyst extension could not be checked out.')
        raise #return

except:
    arcpy.AddError('Could not generate kernel density.')
    raise # return
    